In [1]:
import os
import numpy as np
import pandas as pd
import re


## Configurations

In [2]:
normal_dataset_path = '../../datasets/posenet-uncut/'
ugly_dataset_path = '../../datasets/good_ugly_posenet/'
subjective_score = pd.read_csv('../../datasets/VideoScoring.csv')

all_data = None

In [3]:
def add_scores(file_name,ugly=False):
    path = normal_dataset_path
    prefix = ''
    
    if ugly:
        path = ugly_dataset_path
        prefix = 'U' 
        
    df = pd.read_csv(path+file_name)
    df['ConfidenceScore'] = df[list(df.filter(regex='_score'))].mean(axis=1) * df['score']
    sub_score = subjective_score.loc[subjective_score['FileName'] == prefix + file_name.replace('.csv', '')]['AVG']
    df['GoodnessScore'] = int(sub_score)
    
    return df

## Add Scores

In [4]:
# Add scores to the original videos
for file in os.listdir(normal_dataset_path):
    if not file.find(".csv",0) == -1:
        df = add_scores(file,False)
        
        if all_data is None:
            all_data = df
        else:
            all_data = all_data.append(df, ignore_index=True)  

# Add scores to augmented videos
for file in os.listdir(ugly_dataset_path):
    if not file.find(".csv",0) == -1:
        df = add_scores(file,True)
        
        if all_data is None:
            all_data = df
        else:
            all_data = all_data.append(df, ignore_index=True)  

## Save result

In [5]:
slice_size = 4
cut_size = int(len(all_data) / slice_size)
q = 0

for i in range(slice_size):
    temp = all_data.loc[cut_size*i:cut_size*(i+1)-1,:]
    temp.to_csv('../../datasets/posenet_scored_all_{}.csv'.format(i+1), index=False)
    
    
print('A dataset with {} has been saved in /datasets/posenet_scored_all.csv'.format(len(all_data)))

A dataset with 225092 has been saved in /datasets/posenet_scored_all.csv
